In [ ]:
import pandas as pd
import numpy as np, os
import matplotlib.pyplot as plt

from pathlib import Path
# import yaml
# import re

# import pyarrow as pa
# import pyarrow.parquet as pq

# import gbd_mapping as gm
from vivarium import InteractiveContext
import scipy

# from db_queries import get_ids, get_outputs, get_population, get_covariate_estimates
# from get_draws.api import get_draws

# import vivarium_helpers as vh
# import vivarium_helpers.id_helper as idh
# from vivarium_helpers.vph_output.operations import VPHOperator
# from vivarium_helpers.vph_output import cleaning

!date
!whoami
!pwd

Mon Oct 20 14:26:52 PDT 2025
ndbs
/mnt/share/code/ndbs/vivarium_research_alzheimers/verification_and_validation


In [ ]:
! pip list | grep vivarium

vivarium                  3.5.5
vivarium_build_utils      2.0.11
vivarium_cluster_tools    2.1.21
vivarium_csu_alzheimers   0.1.dev49+gc7315b070 /mnt/share/code/ndbs/vivarium_csu_alzheimers
vivarium_dependencies     1.0.1
vivarium_public_health    4.3.12


In [ ]:
! pip freeze | grep vivarium

vivarium==3.5.5
vivarium_build_utils==2.0.11
vivarium_cluster_tools==2.1.21
-e git+https://github.com/ihmeuw/vivarium_csu_alzheimers.git@c7315b0703d0e097dce6e7600f2ac2846d4a4e31#egg=vivarium_csu_alzheimers
vivarium_dependencies==1.0.1
vivarium_public_health==4.3.12


# Automatically find the path to the model spec file

Code from [Vivarium Research documentation of interactive
simulations](https://vivarium-research.readthedocs.io/en/latest/model_design/designing_vivarium_model/interactive_sim/index.html).

See also the [interactive simulation tutorial in the Vivarium docs](https://vivarium.readthedocs.io/en/latest/tutorials/running_a_simulation/interactive.html).

In [95]:
import vivarium_csu_alzheimers
# model_specifications/model_spec.yaml is the typical location within the engineering repo for the
# default spec to live. Ask the engineers if you can't find it.
path = Path(vivarium_csu_alzheimers.__file__.replace('__init__.py', 'model_specifications/model_spec.yaml'))
print(path)

!cat $path

/mnt/share/code/ndbs/vivarium_csu_alzheimers/src/vivarium_csu_alzheimers/model_specifications/model_spec.yaml
components:
    vivarium_csu_alzheimers:
      components:
          - ResultsStratifier()
          - NewSimulantsObserver()
          - AlzheimersIncidence()
          - AlzheimersPopulation('population.scaling_factor')
          - Alzheimers()
          - Testing()
          - BaselineTestingObserver()
          - BBBMTestingObserver()
          - Treatment()
          - TreatmentObserver()
          - TreatmentRiskEffect('cause.alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate')
            
    vivarium_public_health:
        population:
            - Mortality()
        results:
            - DisabilityObserver()
            - MortalityObserver()
            - DiseaseObserver('alzheimers_disease_and_other_dementias')

configuration:
    input_data:
        input_draw_number: 0
        artifact_path: '/mnt/team/simulation_

# Customize the model specification by deleting observers

In [96]:
from vivarium.framework.configuration import build_model_specification
custom_model_specification = build_model_specification(path)

# Delete mortality, morbidity and disease observers
print(custom_model_specification.components.vivarium_public_health.results)
del custom_model_specification.components.vivarium_public_health.results

# Filter out components with 'Observer' or 'Stratifier' in the name
observers = [
    c for c in custom_model_specification.components.vivarium_csu_alzheimers.components
    if 'Observer' in c or 'Stratifier' in c
]
print(observers)
custom_model_specification.components.vivarium_csu_alzheimers.components = [
    c for c in custom_model_specification.components.vivarium_csu_alzheimers.components
    if c not in observers
]

# Set scenario to intervention instead of baseline
custom_model_specification.configuration.intervention.scenario = 'bbbm_testing_and_treatment'

custom_model_specification.components.vivarium_csu_alzheimers.components


['DisabilityObserver()', 'MortalityObserver()', "DiseaseObserver('alzheimers_disease_and_other_dementias')"]
['ResultsStratifier()', 'NewSimulantsObserver()', 'BaselineTestingObserver()', 'BBBMTestingObserver()', 'TreatmentObserver()']


['AlzheimersIncidence()',
 "AlzheimersPopulation('population.scaling_factor')",
 'Alzheimers()',
 'Testing()',
 'Treatment()',
 "TreatmentRiskEffect('cause.alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate')"]

# Here's an alternative way to get a model spec and create a sim

In [ ]:
location = "United States of America"
project_directory = Path("/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/")
model_spec_path = (
    project_directory /
    "results/model7.0/model_spec/2025_10_09_17_56_46/model_specification.yaml"
)
artifact_path = (
    project_directory /
    f"artifacts/model7.0/{location.lower().replace(' ', '_')}.hdf"
)

sim = InteractiveContext(model_spec_path, setup=False)
sim.configuration.update(
    {
        'input_data': {'artifact_path': artifact_path},
        'intervention': {'scenario': 'bbbm_testing_and_treatment'},
    }
)
sim.setup()

2025-10-20 16:14:23.340 | INFO     | simulation_3-artifact_manager:79 - Running simulation from artifact located at /mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model7.0/united_states_of_america.hdf.
2025-10-20 16:14:23.341 | INFO     | simulation_3-artifact_manager:80 - Artifact base filter terms are ['draw == 0'].
2025-10-20 16:14:23.342 | INFO     | simulation_3-artifact_manager:81 - Artifact additional filter terms are None.
2025-10-20 16:14:25.406 | INFO     | simulation_3-results_context:129 - The following stratifications are registered but not used by any observers: 
['current_year']
2025-10-20 16:14:25.523 | WARNING  | simulation_3-resource_manager:176 - Resource stream.alzheimers_disease_and_other_dementias_initial_states is not produced by any component but is needed to compute (column.alzheimers_disease_and_other_dementias).
2025-10-20 16:14:25.525 | WARNING  | simulation_3-resource_manager:176 - Resource stream.treatment_initial_states is not p

# Create a sim using custom model specification

In [59]:
sim = InteractiveContext(custom_model_specification)

2025-10-20 17:19:58.357 | INFO     | simulation_7-artifact_manager:79 - Running simulation from artifact located at /mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model7.0/united_states_of_america.hdf.
2025-10-20 17:19:58.358 | INFO     | simulation_7-artifact_manager:80 - Artifact base filter terms are ['draw == 0'].
2025-10-20 17:19:58.359 | INFO     | simulation_7-artifact_manager:81 - Artifact additional filter terms are None.
2025-10-20 17:20:01.065 | WARNING  | simulation_7-values_manager:55 - Unsourced pipelines: ['all_causes.disability_weight']
2025-10-20 17:20:01.068 | WARNING  | simulation_7-resource_manager:176 - Resource stream.alzheimers_disease_and_other_dementias_initial_states is not produced by any component but is needed to compute (column.alzheimers_disease_and_other_dementias).
2025-10-20 17:20:01.069 | WARNING  | simulation_7-resource_manager:176 - Resource stream.treatment_initial_states is not produced by any component but is needed to 

In [60]:
pop = sim.get_population()
pop

,tracked,entrance_time,sex,exit_time,location,age,alive,cause_of_death,years_of_life_lost,alzheimers_disease_and_other_dementias,...,waning_effect_long_event_time,waning_effect_long_event_count,waning_effect_short_event_count,waning_effect_short_event_time,no_effect_after_long_event_count,no_effect_after_long_event_time,no_effect_after_short_event_count,no_effect_after_short_event_time,no_effect_never_treated_event_count,no_effect_never_treated_event_time
0,True,2024-07-03,Male,NaT,United States of America,90.089181,alive,not_dead,0.0,alzheimers_blood_based_biomarker_state,...,NaT,0,0,NaT,0,NaT,0,NaT,0,NaT
1,True,2024-07-03,Male,NaT,United States of America,92.196689,alive,not_dead,0.0,alzheimers_disease_state,...,NaT,0,0,NaT,0,NaT,0,NaT,0,NaT
2,True,2024-07-03,Female,NaT,United States of America,71.338013,alive,not_dead,0.0,alzheimers_blood_based_biomarker_state,...,NaT,0,0,NaT,0,NaT,0,NaT,0,NaT
3,True,2024-07-03,Male,NaT,United States of America,83.122725,alive,not_dead,0.0,alzheimers_mild_cognitive_impairment_state,...,NaT,0,0,NaT,0,NaT,0,NaT,0,NaT
4,True,2024-07-03,Female,NaT,United States of America,87.738486,alive,not_dead,0.0,alzheimers_blood_based_biomarker_state,...,NaT,0,0,NaT,0,NaT,0,NaT,0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,True,2024-07-03,Female,NaT,United States of America,79.392384,alive,not_dead,0.0,alzheimers_blood_based_biomarker_state,...,NaT,0,0,NaT,0,NaT,0,NaT,0,NaT
19996,True,2024-07-03,Female,NaT,United States of America,85.789441,alive,not_dead,0.0,alzheimers_blood_based_biomarker_state,...,NaT,0,0,NaT,0,NaT,0,NaT,0,NaT
19997,True,2024-07-03,Female,NaT,United States of America,85.792091,alive,not_dead,0.0,alzheimers_blood_based_biomarker_state,...,NaT,0,0,NaT,0,NaT,0,NaT,0,NaT
19998,True,2024-07-03,Female,NaT,United States of America,75.836127,alive,not_dead,0.0,alzheimers_disease_state,...,NaT,0,0,NaT,0,NaT,0,NaT,0,NaT


# Print list of columns and list of pipeline values

In [45]:
pop.columns

Index(['tracked', 'entrance_time', 'sex', 'exit_time', 'location', 'age',
       'alive', 'cause_of_death', 'years_of_life_lost',
       'alzheimers_disease_and_other_dementias', 'treatment',
       'bbbm_entrance_time', 'previous_alzheimers_disease_and_other_dementias',
       'alzheimers_blood_based_biomarker_state_event_time',
       'alzheimers_blood_based_biomarker_state_event_count',
       'alzheimers_mild_cognitive_impairment_state_event_count',
       'alzheimers_mild_cognitive_impairment_state_event_time',
       'alzheimers_disease_state_event_count',
       'alzheimers_disease_state_event_time', 'bbbm_test_ever_eligible',
       'bbbm_test_date', 'testing_propensity', 'testing_state',
       'bbbm_test_result', 'treatment_propensity',
       'susceptible_to_treatment_event_count',
       'susceptible_to_treatment_event_time', 'positive_test_event_time',
       'positive_test_event_count', 'start_treatment_event_count',
       'start_treatment_event_time', 'full_effect_long_

In [19]:
sim.list_values()

['simulant_step_size',
 'cause_specific_mortality_rate',
 'alzheimers_blood_based_biomarker_state.dwell_time',
 'alzheimers_blood_based_biomarker_state.disability_weight',
 'all_causes.disability_weight',
 'alzheimers_blood_based_biomarker_state.excess_mortality_rate.paf',
 'alzheimers_blood_based_biomarker_state.excess_mortality_rate',
 'mortality_rate',
 'alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate.paf',
 'alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate',
 'alzheimers_mild_cognitive_impairment_state.dwell_time',
 'alzheimers_mild_cognitive_impairment_state.disability_weight',
 'alzheimers_mild_cognitive_impairment_state.excess_mortality_rate.paf',
 'alzheimers_mild_cognitive_impairment_state.excess_mortality_rate',
 'alzheimers_mild_cognitive_impairment_state_to_alzheimers_disease_state.transition_rate.paf',
 'alzheimers_mild_cognitive_impairment_state_to_alzheimers_disease_s

In [65]:
[v for v in sim.list_values() if 'rate' in v]

['cause_specific_mortality_rate',
 'alzheimers_blood_based_biomarker_state.excess_mortality_rate.paf',
 'alzheimers_blood_based_biomarker_state.excess_mortality_rate',
 'mortality_rate',
 'alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate.paf',
 'alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate',
 'alzheimers_mild_cognitive_impairment_state.excess_mortality_rate.paf',
 'alzheimers_mild_cognitive_impairment_state.excess_mortality_rate',
 'alzheimers_mild_cognitive_impairment_state_to_alzheimers_disease_state.transition_rate.paf',
 'alzheimers_mild_cognitive_impairment_state_to_alzheimers_disease_state.transition_rate',
 'alzheimers_disease_state.excess_mortality_rate.paf',
 'alzheimers_disease_state.excess_mortality_rate',
 'start_treatment.excess_mortality_rate.paf',
 'start_treatment.excess_mortality_rate',
 'full_effect_long.excess_mortality_rate.paf',
 'full_effect_long.excess_mort

In [83]:
pop['alzheimers_disease_and_other_dementias']

0            alzheimers_blood_based_biomarker_state
1                          alzheimers_disease_state
2                          alzheimers_disease_state
3        alzheimers_mild_cognitive_impairment_state
4        alzheimers_mild_cognitive_impairment_state
                            ...                    
78250                      alzheimers_disease_state
78251    alzheimers_mild_cognitive_impairment_state
78252        alzheimers_blood_based_biomarker_state
78253        alzheimers_blood_based_biomarker_state
78254                      alzheimers_disease_state
Name: alzheimers_disease_and_other_dementias, Length: 78255, dtype: object

In [139]:
s = sim.get_value('alzheimers_mild_cognitive_impairment_state.dwell_time')(pop.index)
s.loc[s>0]

Series([], Name: alzheimers_mild_cognitive_impairment_state.dwell_time, dtype: float64)

In [144]:
s = sim.get_value('alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate.paf')(pop.index)
s.loc[s>0]

Series([], Name: 0, dtype: float64)

# Advance the population to a time when some people have treatment

In [61]:
sim.run_until(pd.Timestamp('2040-12-31'))

2025-10-20 17:20:54.130 | INFO     | simulation_7 - vivarium.framework.engine:284 - 2025-01-01 00:00:00
2025-10-20 17:20:58.038 | INFO     | simulation_7 - vivarium.framework.engine:284 - 2025-07-02 00:00:00
2025-10-20 17:21:02.148 | INFO     | simulation_7 - vivarium.framework.engine:284 - 2025-12-31 00:00:00
2025-10-20 17:21:06.356 | INFO     | simulation_7 - vivarium.framework.engine:284 - 2026-07-01 00:00:00
2025-10-20 17:21:10.798 | INFO     | simulation_7 - vivarium.framework.engine:284 - 2026-12-30 00:00:00
2025-10-20 17:21:15.628 | INFO     | simulation_7 - vivarium.framework.engine:284 - 2027-06-30 00:00:00
2025-10-20 17:21:20.123 | INFO     | simulation_7 - vivarium.framework.engine:284 - 2027-12-29 00:00:00
2025-10-20 17:21:25.130 | INFO     | simulation_7 - vivarium.framework.engine:284 - 2028-06-28 00:00:00
2025-10-20 17:21:30.011 | INFO     | simulation_7 - vivarium.framework.engine:284 - 2028-12-27 00:00:00
2025-10-20 17:21:34.569 | INFO     | simulation_7 - vivarium.fra

In [63]:
pop = sim.get_population()
pop

,tracked,entrance_time,sex,exit_time,location,age,alive,cause_of_death,years_of_life_lost,alzheimers_disease_and_other_dementias,...,waning_effect_long_event_time,waning_effect_long_event_count,waning_effect_short_event_count,waning_effect_short_event_time,no_effect_after_long_event_count,no_effect_after_long_event_time,no_effect_after_short_event_count,no_effect_after_short_event_time,no_effect_never_treated_event_count,no_effect_never_treated_event_time
0,True,2024-07-03,Male,2032-12-22,United States of America,97.563514,dead,other_causes,8.215451,alzheimers_blood_based_biomarker_state,...,NaT,0,0,NaT,0,NaT,0,NaT,0,NaT
1,True,2024-07-03,Male,2028-12-27,United States of America,95.684711,dead,other_causes,8.402510,alzheimers_disease_state,...,NaT,0,0,NaT,0,NaT,0,NaT,0,NaT
2,True,2024-07-03,Female,NaT,United States of America,87.781544,alive,not_dead,0.000000,alzheimers_disease_state,...,NaT,0,0,NaT,0,NaT,0,NaT,0,NaT
3,True,2024-07-03,Male,2031-06-25,United States of America,89.102191,dead,other_causes,9.242844,alzheimers_mild_cognitive_impairment_state,...,NaT,0,0,NaT,0,NaT,0,NaT,0,NaT
4,True,2024-07-03,Female,2025-07-02,United States of America,87.738486,dead,other_causes,9.744761,alzheimers_mild_cognitive_impairment_state,...,NaT,0,0,NaT,0,NaT,0,NaT,0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78250,True,2040-12-12,Male,NaT,NaN,89.492027,alive,not_dead,0.000000,alzheimers_disease_state,...,NaT,0,0,NaT,0,NaT,0,NaT,0,NaT
78251,True,2040-12-12,Male,NaT,NaN,86.141478,alive,not_dead,0.000000,alzheimers_mild_cognitive_impairment_state,...,NaT,0,0,NaT,0,NaT,0,NaT,0,NaT
78252,True,2040-12-12,Male,NaT,NaN,88.860750,alive,not_dead,0.000000,alzheimers_blood_based_biomarker_state,...,NaT,0,0,NaT,0,NaT,0,NaT,0,NaT
78253,True,2040-12-12,Male,NaT,NaN,87.154364,alive,not_dead,0.000000,alzheimers_blood_based_biomarker_state,...,NaT,0,0,NaT,0,NaT,0,NaT,0,NaT


In [62]:
sim.get_value('treatment.exposure')(pop.index)

0        susceptible_to_treatment
1        susceptible_to_treatment
2        susceptible_to_treatment
3        susceptible_to_treatment
4        susceptible_to_treatment
                   ...           
19995    susceptible_to_treatment
19996    susceptible_to_treatment
19997    susceptible_to_treatment
19998    susceptible_to_treatment
19999    susceptible_to_treatment
Name: treatment.exposure, Length: 20000, dtype: object

In [ ]:
sim.get_value(
    'treatment_on_alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.relative_risk')(pop.index)

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
78250    1.0
78251    1.0
78252    1.0
78253    1.0
78254    1.0
Name: treatment_on_alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.relative_risk, Length: 78255, dtype: float64

In [85]:
sim.current_time

Timestamp('2046-06-06 00:00:00')

In [148]:
# List columns we want to keep
columns = [
    'age', 'alive', 'bbbm_test_result', 'treatment',
    'alzheimers_disease_and_other_dementias',
    'bbbm_entrance_time',
]
# Rename some columns with shorter names if desired
columns_to_short_names = {c: c for c in columns} | {
    'alzheimers_disease_and_other_dementias': 'ad_state',
}
# Simultaneously list pipeline values we want and map them to shorter
# names
values_to_short_names = {
    'alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate':
        'mci_inc_rate',
    'treatment_on_alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.relative_risk':
        'relative_risk',
}

def get_pop_columns(
        sim,
        columns_to_short_names=columns_to_short_names,
        values_to_short_names=values_to_short_names,
    ):
    """Get the specified subset of population columns concatenated with
    the specified pipeline values, with columns for pipeline values
    renamed according to the names in the provided "short names"
    dictionary.
    """
    pop = sim.get_population()
    pipeline_values = [
        sim.get_value(v)(pop.index).rename(short_name)
        for v, short_name in values_to_short_names.items()]
    pop_columns = (
        pop[columns_to_short_names.keys()]
        .rename(columns=columns_to_short_names)
        .join(pipeline_values)
        # Add current time to the dataframe
        .assign(current_time=sim.current_time)
    )
    return pop_columns

def take_steps(
        sim,
        n,
        columns_to_short_names=columns_to_short_names,
        values_to_short_names=values_to_short_names,
    ):
    """Take n simulation steps, recording the specified population
    columns and values at each step, and return a list of the population
    tables at each step. The resulting list will have n+1 population
    tables.
    """
    population_trace = [
        get_pop_columns(sim, columns_to_short_names, values_to_short_names)]
    for _ in range(n):
        sim.step()
        population_trace.append(get_pop_columns(
            sim, columns_to_short_names, values_to_short_names))
    return population_trace

pop1 = get_pop_columns(sim)
pop1

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
0,97.563514,dead,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2022-08-03 07:39:38.969251390,0.060274,1.0,2048-06-03
1,95.684711,dead,not_tested,susceptible_to_treatment,alzheimers_disease_state,2023-07-19 05:22:18.080798424,0.059775,1.0,2048-06-03
2,94.757588,alive,not_tested,susceptible_to_treatment,alzheimers_disease_state,2022-04-10 17:02:42.280338871,0.060434,1.0,2048-06-03
3,89.102191,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2022-03-11 15:55:47.288939655,0.060476,1.0,2048-06-03
4,87.738486,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2020-10-12 08:00:15.290018928,0.061177,1.0,2048-06-03
...,...,...,...,...,...,...,...,...,...
105553,89.091235,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-12-04 00:00:00.000000000,0.025287,1.0,2048-06-03
105554,86.745849,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-12-04 00:00:00.000000000,0.025287,1.0,2048-06-03
105555,87.017824,alive,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2047-12-04 00:00:00.000000000,0.025287,1.0,2048-06-03
105556,87.445953,alive,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2047-12-04 00:00:00.000000000,0.025287,1.0,2048-06-03


In [77]:
pop1.query("relative_risk != 1.0")

,age,alive,bbbm_test_result,treatment,mci_inc_rate,relative_risk
146,87.648047,alive,positive,waning_effect_long,0.046120,0.842505
360,86.536714,dead,positive,waning_effect_long,0.046734,0.842505
618,77.842615,alive,positive,waning_effect_long,0.047869,0.842505
1182,81.964068,alive,positive,waning_effect_long,0.047984,0.842505
1191,73.798045,alive,positive,full_effect_long,0.029184,0.527516
...,...,...,...,...,...,...
74036,77.998849,alive,positive,full_effect_long,0.016986,0.527516
74037,78.233413,alive,positive,full_effect_long,0.016986,0.527516
74039,77.707071,alive,positive,full_effect_long,0.016986,0.527516
74041,80.301554,alive,positive,full_effect_short,0.016986,0.527516


In [97]:
pop_trace = take_steps(sim, 4)
len(pop_trace)

2025-10-20 22:11:43.441 | INFO     | simulation_7 - vivarium.framework.engine:284 - 2046-06-06 00:00:00
2025-10-20 22:11:54.723 | INFO     | simulation_7 - vivarium.framework.engine:284 - 2046-12-05 00:00:00
2025-10-20 22:12:05.703 | INFO     | simulation_7 - vivarium.framework.engine:284 - 2047-06-05 00:00:00
2025-10-20 22:12:17.647 | INFO     | simulation_7 - vivarium.framework.engine:284 - 2047-12-04 00:00:00


5

In [98]:
pop_trace[0]

,age,alive,bbbm_test_result,treatment,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
0,97.563514,dead,not_tested,susceptible_to_treatment,2022-08-03 07:39:38.969251390,0.059218,1.0,2046-06-06
1,95.684711,dead,not_tested,susceptible_to_treatment,2023-07-19 05:22:18.080798424,0.058686,1.0,2046-06-06
2,92.764433,alive,not_tested,susceptible_to_treatment,2022-04-10 17:02:42.280338871,0.059389,1.0,2046-06-06
3,89.102191,dead,not_tested,susceptible_to_treatment,2022-03-11 15:55:47.288939655,0.059433,1.0,2046-06-06
4,87.738486,dead,not_tested,susceptible_to_treatment,2020-10-12 08:00:15.290018928,0.060178,1.0,2046-06-06
...,...,...,...,...,...,...,...,...
97706,88.809023,alive,not_tested,susceptible_to_treatment,2045-12-06 00:00:00.000000000,0.025287,1.0,2046-06-06
97707,88.623685,alive,not_tested,susceptible_to_treatment,2045-12-06 00:00:00.000000000,0.025287,1.0,2046-06-06
97708,86.586960,alive,not_tested,susceptible_to_treatment,2045-12-06 00:00:00.000000000,0.025287,1.0,2046-06-06
97709,87.517808,alive,not_tested,susceptible_to_treatment,2045-12-06 00:00:00.000000000,0.025287,1.0,2046-06-06


In [99]:
pop_trace[1]

,age,alive,bbbm_test_result,treatment,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
0,97.563514,dead,not_tested,susceptible_to_treatment,2022-08-03 07:39:38.969251390,0.059488,1.0,2046-12-05
1,95.684711,dead,not_tested,susceptible_to_treatment,2023-07-19 05:22:18.080798424,0.058965,1.0,2046-12-05
2,93.262722,alive,not_tested,susceptible_to_treatment,2022-04-10 17:02:42.280338871,0.059656,1.0,2046-12-05
3,89.102191,dead,not_tested,susceptible_to_treatment,2022-03-11 15:55:47.288939655,0.059700,1.0,2046-12-05
4,87.738486,dead,not_tested,susceptible_to_treatment,2020-10-12 08:00:15.290018928,0.060433,1.0,2046-12-05
...,...,...,...,...,...,...,...,...
99705,89.681840,alive,not_tested,susceptible_to_treatment,2046-06-06 00:00:00.000000000,0.025287,1.0,2046-12-05
99706,90.017066,alive,not_tested,susceptible_to_treatment,2046-06-06 00:00:00.000000000,0.025287,1.0,2046-12-05
99707,89.424016,alive,not_tested,susceptible_to_treatment,2046-06-06 00:00:00.000000000,0.025287,1.0,2046-12-05
99708,86.227436,alive,not_tested,susceptible_to_treatment,2046-06-06 00:00:00.000000000,0.025287,1.0,2046-12-05


In [100]:
pop_trace[2]

,age,alive,bbbm_test_result,treatment,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
0,97.563514,dead,not_tested,susceptible_to_treatment,2022-08-03 07:39:38.969251390,0.059754,1.0,2047-06-05
1,95.684711,dead,not_tested,susceptible_to_treatment,2023-07-19 05:22:18.080798424,0.059239,1.0,2047-06-05
2,93.761010,alive,not_tested,susceptible_to_treatment,2022-04-10 17:02:42.280338871,0.059920,1.0,2047-06-05
3,89.102191,dead,not_tested,susceptible_to_treatment,2022-03-11 15:55:47.288939655,0.059963,1.0,2047-06-05
4,87.738486,dead,not_tested,susceptible_to_treatment,2020-10-12 08:00:15.290018928,0.060685,1.0,2047-06-05
...,...,...,...,...,...,...,...,...
101650,88.798279,alive,not_tested,susceptible_to_treatment,2046-12-05 00:00:00.000000000,0.025287,1.0,2047-06-05
101651,86.185105,alive,not_tested,susceptible_to_treatment,2046-12-05 00:00:00.000000000,0.025287,1.0,2047-06-05
101652,86.757333,alive,not_tested,susceptible_to_treatment,2046-12-05 00:00:00.000000000,0.025287,1.0,2047-06-05
101653,88.157599,alive,not_tested,susceptible_to_treatment,2046-12-05 00:00:00.000000000,0.025287,1.0,2047-06-05


# Define Weibull distribution and hazard function

In [102]:
λ=6.76
k=1.22

weibull_dist = scipy.stats.weibull_min(k, scale=λ)
weibull_dist.cdf([5, 10])

array([0.49950771, 0.8005858 ])

In [ ]:
def hazard(t, dist):
    # hazard = (probability density) / (survival function)
    return dist.pdf(t) / dist.sf(t)

def hazard_with_logs(t, dist):
    # Take logs first to avoid underflow
    return np.exp(dist.logpdf(t) - dist.logsf(t))

mci_hazard_func = lambda t: hazard_with_logs(t, weibull_dist)

In [128]:
DAYS_PER_YEAR = 365.25

def years_in_bbbm(current_time, bbbm_entrance_time):
    return (current_time - bbbm_entrance_time).dt.days / DAYS_PER_YEAR

def mci_incidence_hazard(pop):
    hazard = mci_hazard_func(
        years_in_bbbm(pop['current_time'], pop['bbbm_entrance_time']))
    return pd.Series(hazard, index=pop.index, name='mci_hazard')

mci_incidence_hazard(pop_trace[1])

0        0.239223
1        0.237118
2        0.239900
3        0.240076
4        0.243022
           ...   
99705    0.101688
99706    0.101688
99707    0.101688
99708    0.101688
99709    0.101688
Name: mci_hazard, Length: 99710, dtype: float64

# Check that calculated Weibull hazard is increasing over time

In [130]:
pd.concat([mci_incidence_hazard(pop) * pop['relative_risk'] for pop in pop_trace], axis=1)

,0,1,2,3,4
0,0.238137,0.239223,0.240292,0.241344,0.242381
1,0.235997,0.237118,0.238221,0.239306,0.240373
2,0.238825,0.239900,0.240959,0.242001,0.243028
3,0.239003,0.240076,0.241132,0.242171,0.243195
4,0.241995,0.243022,0.244034,0.245030,0.246013
...,...,...,...,...,...
105553,NaN,NaN,NaN,NaN,0.101688
105554,NaN,NaN,NaN,NaN,0.101688
105555,NaN,NaN,NaN,NaN,0.101688
105556,NaN,NaN,NaN,NaN,0.101688


In [134]:
pops = [pop.assign(mci_hazard=mci_incidence_hazard(pop)) for pop in pop_trace]
pops[0]

,age,alive,bbbm_test_result,treatment,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard
0,97.563514,dead,not_tested,susceptible_to_treatment,2022-08-03 07:39:38.969251390,0.059218,1.0,2046-06-06,0.238137
1,95.684711,dead,not_tested,susceptible_to_treatment,2023-07-19 05:22:18.080798424,0.058686,1.0,2046-06-06,0.235997
2,92.764433,alive,not_tested,susceptible_to_treatment,2022-04-10 17:02:42.280338871,0.059389,1.0,2046-06-06,0.238825
3,89.102191,dead,not_tested,susceptible_to_treatment,2022-03-11 15:55:47.288939655,0.059433,1.0,2046-06-06,0.239003
4,87.738486,dead,not_tested,susceptible_to_treatment,2020-10-12 08:00:15.290018928,0.060178,1.0,2046-06-06,0.241995
...,...,...,...,...,...,...,...,...,...
97706,88.809023,alive,not_tested,susceptible_to_treatment,2045-12-06 00:00:00.000000000,0.025287,1.0,2046-06-06,0.101688
97707,88.623685,alive,not_tested,susceptible_to_treatment,2045-12-06 00:00:00.000000000,0.025287,1.0,2046-06-06,0.101688
97708,86.586960,alive,not_tested,susceptible_to_treatment,2045-12-06 00:00:00.000000000,0.025287,1.0,2046-06-06,0.101688
97709,87.517808,alive,not_tested,susceptible_to_treatment,2045-12-06 00:00:00.000000000,0.025287,1.0,2046-06-06,0.101688


In [135]:
pops[0].query("alive=='alive'")

,age,alive,bbbm_test_result,treatment,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard
2,92.764433,alive,not_tested,susceptible_to_treatment,2022-04-10 17:02:42.280338871,0.059389,1.0,2046-06-06,0.238825
15,92.713808,alive,not_tested,susceptible_to_treatment,2020-03-19 12:02:19.609739255,0.060468,1.0,2046-06-06,0.243162
42,93.878693,alive,not_tested,susceptible_to_treatment,2024-12-21 15:54:29.991097765,0.057861,1.0,2046-06-06,0.232678
46,93.959501,alive,not_tested,susceptible_to_treatment,2023-08-01 09:47:13.645188902,0.058666,1.0,2046-06-06,0.235916
66,77.000517,alive,not_tested,susceptible_to_treatment,2021-04-01 14:08:54.136587030,0.059934,1.0,2046-06-06,0.241017
...,...,...,...,...,...,...,...,...,...
97706,88.809023,alive,not_tested,susceptible_to_treatment,2045-12-06 00:00:00.000000000,0.025287,1.0,2046-06-06,0.101688
97707,88.623685,alive,not_tested,susceptible_to_treatment,2045-12-06 00:00:00.000000000,0.025287,1.0,2046-06-06,0.101688
97708,86.586960,alive,not_tested,susceptible_to_treatment,2045-12-06 00:00:00.000000000,0.025287,1.0,2046-06-06,0.101688
97709,87.517808,alive,not_tested,susceptible_to_treatment,2045-12-06 00:00:00.000000000,0.025287,1.0,2046-06-06,0.101688


In [136]:
pops[1].query("alive=='alive'")

,age,alive,bbbm_test_result,treatment,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard
2,93.262722,alive,not_tested,susceptible_to_treatment,2022-04-10 17:02:42.280338871,0.059656,1.0,2046-12-05,0.239900
15,93.212097,alive,not_tested,susceptible_to_treatment,2020-03-19 12:02:19.609739255,0.060719,1.0,2046-12-05,0.244171
42,94.376981,alive,not_tested,susceptible_to_treatment,2024-12-21 15:54:29.991097765,0.058154,1.0,2046-12-05,0.233857
46,94.457789,alive,not_tested,susceptible_to_treatment,2023-08-01 09:47:13.645188902,0.058945,1.0,2046-12-05,0.237039
66,77.498806,alive,not_tested,susceptible_to_treatment,2021-04-01 14:08:54.136587030,0.060193,1.0,2046-12-05,0.242058
...,...,...,...,...,...,...,...,...,...
99705,89.681840,alive,not_tested,susceptible_to_treatment,2046-06-06 00:00:00.000000000,0.025287,1.0,2046-12-05,0.101688
99706,90.017066,alive,not_tested,susceptible_to_treatment,2046-06-06 00:00:00.000000000,0.025287,1.0,2046-12-05,0.101688
99707,89.424016,alive,not_tested,susceptible_to_treatment,2046-06-06 00:00:00.000000000,0.025287,1.0,2046-12-05,0.101688
99708,86.227436,alive,not_tested,susceptible_to_treatment,2046-06-06 00:00:00.000000000,0.025287,1.0,2046-12-05,0.101688


In [141]:
pops[1].query("relative_risk < 1")

,age,alive,bbbm_test_result,treatment,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard
1191,79.279222,alive,positive,waning_effect_long,2023-12-12 13:01:32.064051077,0.043322,0.737509,2046-12-05,0.236220
1223,68.324801,dead,positive,full_effect_long,2021-05-27 23:42:15.333880944,0.031711,0.527516,2046-12-05,0.241739
1569,71.575142,alive,positive,waning_effect_long,2022-12-26 19:22:31.964918220,0.049938,0.842505,2046-12-05,0.238359
3854,86.865973,alive,positive,waning_effect_long,2023-12-29 19:57:09.720606096,0.049468,0.842505,2046-12-05,0.236115
4717,85.988723,dead,positive,waning_effect_long,2021-08-28 14:42:11.002134864,0.058407,0.973751,2046-12-05,0.241207
...,...,...,...,...,...,...,...,...,...
95331,79.480713,alive,positive,full_effect_long,2045-06-07 00:00:00.000000000,0.016986,0.527516,2046-12-05,0.129490
95332,80.497856,alive,positive,full_effect_long,2045-06-07 00:00:00.000000000,0.016986,0.527516,2046-12-05,0.129490
95348,80.461014,alive,positive,full_effect_long,2045-06-07 00:00:00.000000000,0.016986,0.527516,2046-12-05,0.129490
95356,78.827736,alive,positive,full_effect_long,2045-06-07 00:00:00.000000000,0.016986,0.527516,2046-12-05,0.129490


In [146]:
pops[1].query("relative_risk > 1 ")

,age,alive,bbbm_test_result,treatment,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard
146,88.644624,dead,positive,waning_effect_long,2024-10-07 15:06:10.903031519,0.065921,1.131246,2046-12-05,0.234336
360,86.536714,dead,positive,waning_effect_long,2023-09-25 20:09:56.556607170,0.066586,1.131246,2046-12-05,0.236701
1182,83.957223,dead,positive,waning_effect_long,2021-06-23 11:57:18.487730747,0.067960,1.131246,2046-12-05,0.241585
1733,84.485867,dead,positive,waning_effect_long,2021-04-06 01:14:28.370333713,0.063346,1.052498,2046-12-05,0.242030
2269,89.230676,dead,positive,waning_effect_long,2021-07-12 15:02:32.659975250,0.067929,1.131246,2046-12-05,0.241476
...,...,...,...,...,...,...,...,...,...
76767,81.427636,dead,positive,waning_effect_short,2040-12-12 00:00:00.000000000,0.060195,1.377987,2046-12-05,0.175667
78357,65.591176,dead,positive,waning_effect_short,2041-06-12 00:00:00.000000000,0.055004,1.283490,2046-12-05,0.172336
78851,77.492753,dead,positive,waning_effect_short,2041-06-12 00:00:00.000000000,0.055004,1.283490,2046-12-05,0.172336
80920,79.884120,dead,positive,waning_effect_short,2041-12-11 00:00:00.000000000,0.045931,1.094497,2046-12-05,0.168760


In [166]:
(pops[1]['mci_hazard'] / pops[1]['mci_inc_rate']).describe()

count    99710.000000
mean         4.085100
std          0.450777
min          1.267211
25%          4.021402
50%          4.021402
75%          4.021402
max          7.623282
dtype: float64

In [168]:
temp = pops[1].query("alive=='alive'")
# (temp['mci_hazard'] / temp['mci_inc_rate']).describe()
temp

,age,alive,bbbm_test_result,treatment,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard
2,93.262722,alive,not_tested,susceptible_to_treatment,2022-04-10 17:02:42.280338871,0.059656,1.0,2046-12-05,0.239900
15,93.212097,alive,not_tested,susceptible_to_treatment,2020-03-19 12:02:19.609739255,0.060719,1.0,2046-12-05,0.244171
42,94.376981,alive,not_tested,susceptible_to_treatment,2024-12-21 15:54:29.991097765,0.058154,1.0,2046-12-05,0.233857
46,94.457789,alive,not_tested,susceptible_to_treatment,2023-08-01 09:47:13.645188902,0.058945,1.0,2046-12-05,0.237039
66,77.498806,alive,not_tested,susceptible_to_treatment,2021-04-01 14:08:54.136587030,0.060193,1.0,2046-12-05,0.242058
...,...,...,...,...,...,...,...,...,...
99705,89.681840,alive,not_tested,susceptible_to_treatment,2046-06-06 00:00:00.000000000,0.025287,1.0,2046-12-05,0.101688
99706,90.017066,alive,not_tested,susceptible_to_treatment,2046-06-06 00:00:00.000000000,0.025287,1.0,2046-12-05,0.101688
99707,89.424016,alive,not_tested,susceptible_to_treatment,2046-06-06 00:00:00.000000000,0.025287,1.0,2046-12-05,0.101688
99708,86.227436,alive,not_tested,susceptible_to_treatment,2046-06-06 00:00:00.000000000,0.025287,1.0,2046-12-05,0.101688


In [152]:
bbbm = pop1.query("alive=='alive' and ad_state=='alzheimers_blood_based_biomarker_state'")
bbbm

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
464,85.359834,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2018-10-16 19:05:04.949560271,0.062119,1.0,2048-06-03
825,79.093705,alive,positive,no_effect_never_treated,alzheimers_blood_based_biomarker_state,2023-01-30 20:06:07.105417295,0.060018,1.0,2048-06-03
865,100.550402,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2024-04-18 07:06:46.843428985,0.059373,1.0,2048-06-03
917,94.562700,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2024-08-10 14:18:23.326592463,0.059203,1.0,2048-06-03
1757,74.744762,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2022-05-29 03:20:41.133313728,0.060366,1.0,2048-06-03
...,...,...,...,...,...,...,...,...,...
105549,85.641037,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-12-04 00:00:00.000000000,0.025287,1.0,2048-06-03
105551,89.459306,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-12-04 00:00:00.000000000,0.025287,1.0,2048-06-03
105553,89.091235,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-12-04 00:00:00.000000000,0.025287,1.0,2048-06-03
105554,86.745849,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-12-04 00:00:00.000000000,0.025287,1.0,2048-06-03


In [154]:
bbbm.treatment.unique()

array(['susceptible_to_treatment', 'no_effect_never_treated',
       'waning_effect_long', 'no_effect_after_long', 'full_effect_long',
       'no_effect_after_short', 'start_treatment', 'waning_effect_short',
       'full_effect_short'], dtype=object)

In [158]:
bbbm_full_effect = bbbm.query("treatment in ['full_effect_long', 'full_effect_short']")
bbbm_full_effect

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
14887,85.322528,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2024-01-05 22:08:29.166587773,0.031401,0.527516,2048-06-03
15879,61.628755,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2021-01-01 10:32:42.365974891,0.032215,0.527516,2048-06-03
20040,80.034388,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2025-01-01 00:00:00.000000000,0.031116,0.527516,2048-06-03
21526,74.795797,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2025-07-02 00:00:00.000000000,0.030970,0.527516,2048-06-03
22941,63.811343,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2025-12-31 00:00:00.000000000,0.030820,0.527516,2048-06-03
...,...,...,...,...,...,...,...,...,...
101286,76.941601,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2046-12-05 00:00:00.000000000,0.016986,0.527516,2048-06-03
101293,80.770392,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2046-12-05 00:00:00.000000000,0.016986,0.527516,2048-06-03
101303,80.025413,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2046-12-05 00:00:00.000000000,0.016986,0.527516,2048-06-03
101306,76.674635,alive,positive,full_effect_short,alzheimers_blood_based_biomarker_state,2046-12-05 00:00:00.000000000,0.016986,0.527516,2048-06-03


In [159]:
bbbm_no_effect = bbbm.query("treatment in ['susceptible_to_treatment', 'no_effect_never_treated', 'no_effect_after_long', 'no_effect_after_short']")
bbbm_no_effect

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time
464,85.359834,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2018-10-16 19:05:04.949560271,0.062119,1.0,2048-06-03
825,79.093705,alive,positive,no_effect_never_treated,alzheimers_blood_based_biomarker_state,2023-01-30 20:06:07.105417295,0.060018,1.0,2048-06-03
865,100.550402,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2024-04-18 07:06:46.843428985,0.059373,1.0,2048-06-03
917,94.562700,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2024-08-10 14:18:23.326592463,0.059203,1.0,2048-06-03
1757,74.744762,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2022-05-29 03:20:41.133313728,0.060366,1.0,2048-06-03
...,...,...,...,...,...,...,...,...,...
105549,85.641037,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-12-04 00:00:00.000000000,0.025287,1.0,2048-06-03
105551,89.459306,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-12-04 00:00:00.000000000,0.025287,1.0,2048-06-03
105553,89.091235,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-12-04 00:00:00.000000000,0.025287,1.0,2048-06-03
105554,86.745849,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2047-12-04 00:00:00.000000000,0.025287,1.0,2048-06-03


In [160]:
bbbm_full_effect.mci_inc_rate.mean() / bbbm_no_effect.mci_inc_rate.mean()

0.5190830966266639

In [165]:
bbbm_full_effect.relative_risk.unique()

array([0.52751591])

# Compare calculated Weibull hazard to hazard rate value from pipeline

In [181]:
ratio = (mci_incidence_hazard(bbbm_no_effect) / bbbm_no_effect['mci_inc_rate']).rename('haz_to_prob_ratio')
ratio.describe()

count    14024.000000
mean         4.021402
std          0.000004
min          4.021305
25%          4.021402
50%          4.021402
75%          4.021402
max          4.021402
Name: haz_to_prob_ratio, dtype: float64

In [183]:
bbbm_no_effect.join(mci_incidence_hazard(bbbm_no_effect)).join(ratio).loc[ratio < 4.021402]

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard,haz_to_prob_ratio
464,85.359834,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2018-10-16 19:05:04.949560271,0.062119,1.0,2048-06-03,0.249806,4.021386
825,79.093705,alive,positive,no_effect_never_treated,alzheimers_blood_based_biomarker_state,2023-01-30 20:06:07.105417295,0.060018,1.0,2048-06-03,0.241356,4.021387
865,100.550402,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2024-04-18 07:06:46.843428985,0.059373,1.0,2048-06-03,0.238759,4.021332
917,94.562700,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2024-08-10 14:18:23.326592463,0.059203,1.0,2048-06-03,0.238076,4.021361
1757,74.744762,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2022-05-29 03:20:41.133313728,0.060366,1.0,2048-06-03,0.242753,4.021322
...,...,...,...,...,...,...,...,...,...,...,...
18614,75.839770,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2021-02-23 14:42:51.363081331,0.060997,1.0,2048-06-03,0.245291,4.021368
18670,100.033083,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2019-07-04 06:11:00.299980092,0.061787,1.0,2048-06-03,0.248468,4.021340
18736,98.475765,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2021-11-04 18:49:24.918056129,0.060651,1.0,2048-06-03,0.243901,4.021383
19020,92.452055,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2024-10-27 10:12:08.045352095,0.059086,1.0,2048-06-03,0.237605,4.021343


In [180]:
ratio = (mci_incidence_hazard(bbbm) /  bbbm['mci_inc_rate']).rename('haz_to_prob_ratio')
bbbm.join(mci_incidence_hazard(bbbm)).join(ratio).loc[ratio > 5]

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,mci_inc_rate,relative_risk,current_time,mci_hazard,haz_to_prob_ratio
13589,89.808865,alive,positive,waning_effect_long,alzheimers_blood_based_biomarker_state,2024-08-19 21:33:05.196899471,0.045206,0.763758,2048-06-03,0.238022,5.265270
14887,85.322528,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2024-01-05 22:08:29.166587773,0.031401,0.527516,2048-06-03,0.239377,7.623267
15879,61.628755,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2021-01-01 10:32:42.365974891,0.032215,0.527516,2048-06-03,0.245578,7.623188
18111,89.947875,alive,positive,waning_effect_long,alzheimers_blood_based_biomarker_state,2022-12-10 13:28:22.197016352,0.045895,0.763758,2048-06-03,0.241648,5.265229
20040,80.034388,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2025-01-01 00:00:00.000000000,0.031116,0.527516,2048-06-03,0.237210,7.623282
...,...,...,...,...,...,...,...,...,...,...,...
101286,76.941601,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2046-12-05 00:00:00.000000000,0.016986,0.527516,2048-06-03,0.129490,7.623282
101293,80.770392,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2046-12-05 00:00:00.000000000,0.016986,0.527516,2048-06-03,0.129490,7.623282
101303,80.025413,alive,positive,full_effect_long,alzheimers_blood_based_biomarker_state,2046-12-05 00:00:00.000000000,0.016986,0.527516,2048-06-03,0.129490,7.623282
101306,76.674635,alive,positive,full_effect_short,alzheimers_blood_based_biomarker_state,2046-12-05 00:00:00.000000000,0.016986,0.527516,2048-06-03,0.129490,7.623282
